# Obtener Publico Objetivo

El primer filtro para obtener al público objetivo es seleccionar a los usuarios que:
- Tienen 65 años o más
- Los movimientos bancarios se centran en gastos en salud y medicamentos (no se incluyen gastos en restaurantes, ocio, etc.)

Además de movimientos que puedan resultar de interés durante el proceso de análisis.

In [4]:
# Importación de librerías
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
# Lectura de datos
df_catalogo_giros = pd.read_csv('../data/Catalogo_Giros.csv')
df_catalogo_giros.head()

,NU_AFILIACION,CD_GIRO,NB_GIRO,NB_SUBGIRO
0,7715643.0,5812.0,RESTAURANTES,"RESTAURANTES,"
1,4208483.0,5812.0,RESTAURANTES,"RESTAURANTES,"
2,4411062.0,5812.0,RESTAURANTES,"RESTAURANTES,"
3,5179627.0,4814.0,TELECOMUNICACIONES,SERVICIOS
4,4064129.0,5912.0,FARMACIAS,"DROGUERIAS,"


In [10]:
df_clientes_descriptivos = pd.read_csv('../data/Clientes_Descriptivo.csv')
df_clientes_descriptivos.head()

,NU_CTE_COD,EDAD,CD_ESTADO,CD_POSTAL,CD_SEXO
0,ABK0NHDA,68.0,DF,13360.0,M
1,M5JVN30B,22.0,JA,44760.0,M
2,RMMTWKSD,22.0,AG,20344.0,M
3,1VLE6Y9G,79.0,CH,32390.0,M
4,3CE1LE8Z,64.0,DU,34414.0,F


In [11]:
df_transacciones_clientes = pd.read_csv('../data/Transacciones_Clientes.csv')
df_transacciones_clientes.head()

,NU_CTE_COD,FH_CORTE,FH_OPERACION,NU_AFILIACION,TIPO_TARJETA,IM_TRANSACCION
0,0MNXBHXN,01/07/2022,12/07/2022,4149168.0,CREDITO,495.00
1,562NS6XR,01/08/2022,03/08/2022,7476840.0,CREDITO,294.00
2,LYS00B64,01/04/2022,13/04/2022,8427123.0,DEBITO,570.00
3,YD17U634,01/05/2022,14/05/2022,6356919.0,DEBITO,301.00
4,M8V2EQUE,01/04/2022,22/04/2022,7979128.0,DEBITO,393.65


### Obtención de datos de interés

In [52]:
# Obtener todos los clientes con 65 años o más
df_clientes_descriptivos_edad = df_clientes_descriptivos[df_clientes_descriptivos['EDAD'] >= 65]
df_clientes_descriptivos_edad.head()

,NU_CTE_COD,EDAD,CD_ESTADO,CD_POSTAL,CD_SEXO
0,ABK0NHDA,68.0,DF,13360.0,M
3,1VLE6Y9G,79.0,CH,32390.0,M
26,TDBWYRTZ,77.0,BN,22180.0,M
61,0VRW8KB0,68.0,NL,67174.0,F
65,0B199MS8,81.0,PU,72310.0,F


In [50]:
# CONNECT TO DATABASE
import mysql.connector

conn = mysql.connector.connect(
    host = 'localhost',
    port = '3307',
    database = 'hackathon_bbva',
    user = 'root',
    password = 'adminroot'
)


In [55]:
# OBTENCION DE GIROS DE INTERÉS

query = """
SELECT DISTINCT NB_GIRO, NB_SUBGIRO 
FROM catalogo_giros
WHERE
	NB_GIRO = 'FARMACIAS'
OR
	NB_SUBGIRO = 'DOCTORES, OTRAS ESPECIALIDADES,(MEDICOS)'
OR
	NB_SUBGIRO = 'OPTOMETRISTAS, (OFTALMOLOGOS)'
OR
	NB_SUBGIRO = 'LABORATORIOS DENTALES, (LABORATORIOS MEDICOS)'
OR
	NB_SUBGIRO = 'MEDICINAS, MEDICINAS DE PATENTE, FARMACOS DIVERSOS'
OR
	NB_SUBGIRO = 'OPTICAS, LENTES Y ANTEOJOS'
OR
	NB_SUBGIRO = 'SERVICIOS MEDICOS Y PARAMEDICOS   NO CLASIFICADOS'
ORDER BY NB_GIRO ASC;
"""

df_catalogo_giros = pd.read_sql(query, conn)
df_catalogo_giros.head()

C:\Users\joshr\AppData\Roaming\Python\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,NB_GIRO,NB_SUBGIRO
0,FARMACIAS,"DROGUERIAS,"
1,MEDICOS Y DENTISTAS,"DOCTORES, OTRAS ESPECIALIDADES,(MEDICOS)"
2,MEDICOS Y DENTISTAS,"OPTOMETRISTAS, (OFTALMOLOGOS)"
3,OTROS,"LABORATORIOS DENTALES, (LABORATORIOS MEDICOS)"
4,OTROS,"MEDICINAS, MEDICINAS DE PATENTE, FARMACOS DIVE..."


In [61]:
# CREAR UN CSV CON LOS GIROS y SUBGIROS DE INTERÉS
giros_interes = df_catalogo_giros['NB_GIRO']
subgiros_interes = df_catalogo_giros['NB_SUBGIRO']

df_giros_interes = pd.DataFrame({'NB_GIRO': giros_interes, 'NB_SUBGIRO': subgiros_interes})
df_giros_interes.to_csv('../data/Giros_Interes.csv', index=False)